# Developing a Pangeo-Forge Recipe
First we need to take a look at the dataset structure. Lets look at one of the netcdfs using the download url and fsspec.

In [1]:
import fsspec
import xarray as xr

url = 'https://zenodo.org/record/7761881/files/METAFLUX_GPP_RECO_daily_200101.nc?download=1'
with fsspec.open(url) as f:
   ds = xr.open_dataset(f)

In [2]:
ds

<xarray.Dataset>
Dimensions:   (time: 31, lat: 720, lon: 1440)
Coordinates:
  * time      (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2001-01-31
  * lat       (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.0 89.25 89.5 89.75
  * lon       (lon) float64 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
Data variables:
    GPP       (time, lat, lon) float64 ...
    GPP_std   (time, lat, lon) float64 ...
    RECO      (time, lat, lon) float64 ...
    RECO_std  (time, lat, lon) float64 ...
Attributes:
    description:  Upscaled global product of GPP and Reco (a combination of a...

In [1]:
from feedstock.recipe import transforms

In [2]:
import apache_beam as beam

with beam.Pipeline() as p:
    p | transforms

/Users/juliusbusecke/miniconda/envs/test_leap_dataset/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py:52: UserWarning: Unknown file type specified without xarray engine, backend engine will be automatically selected by xarray
  warnings.warn(
/Users/juliusbusecke/miniconda/envs/test_leap_dataset/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py:52: UserWarning: Unknown file type specified without xarray engine, backend engine will be automatically selected by xarray
  warnings.warn(


In [6]:
import xarray as xr
ds = xr.open_zarr('output/test_leap_dataset.zarr')
ds

<xarray.Dataset>
Dimensions:   (time: 59, lat: 720, lon: 1440)
Coordinates:
  * lat       (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.0 89.25 89.5 89.75
  * lon       (lon) float64 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * time      (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2001-02-28
Data variables:
    GPP       (time, lat, lon) float64 dask.array<chunksize=(31, 720, 1440), meta=np.ndarray>
    GPP_std   (time, lat, lon) float64 dask.array<chunksize=(31, 720, 1440), meta=np.ndarray>
    RECO      (time, lat, lon) float64 dask.array<chunksize=(31, 720, 1440), meta=np.ndarray>
    RECO_std  (time, lat, lon) float64 dask.array<chunksize=(31, 720, 1440), meta=np.ndarray>
Attributes:
    description:  Upscaled global product of GPP and Reco (a combination of a...

Ok this works to write locally, but can I also populate a google cloud bucket from my machine?

In [1]:
!export GOOGLE_APPLICATION_CREDENTIALS=/Users/juliusbusecke/Downloads/keys/leap-pangeo-dataflow_service_account.json

In [2]:
from feedstock.recipe import transforms # this has target_root hardcoded

In [3]:
import apache_beam as beam

with beam.Pipeline() as p:
    p | transforms

/Users/juliusbusecke/miniconda/envs/test_leap_dataset/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py:52: UserWarning: Unknown file type specified without xarray engine, backend engine will be automatically selected by xarray
  warnings.warn(
/Users/juliusbusecke/miniconda/envs/test_leap_dataset/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py:52: UserWarning: Unknown file type specified without xarray engine, backend engine will be automatically selected by xarray
  warnings.warn(


ServerDisconnectedError: Server disconnected [while running '[3]: Create|OpenURLWithFSSpec|OpenWithXarray|StoreToZarr/StoreToZarr/Rechunk/MapTuple(combine_fragments)']

In [3]:
import xarray as xr
ds = xr.open_zarr('gs://leap-persistent/jbusecke/data/library/test/test_leap_dataset.zarr')
ds

<xarray.Dataset>
Dimensions:    (time: 24, latitude: 720, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time       (time) datetime64[ns] 2001-01-01 2001-02-01 ... 2002-12-01
Data variables:
    GPP        (time, latitude, longitude) float64 dask.array<chunksize=(12, 720, 1440), meta=np.ndarray>
    GPP_std    (time, latitude, longitude) float64 dask.array<chunksize=(12, 720, 1440), meta=np.ndarray>
    RECO       (time, latitude, longitude) float64 dask.array<chunksize=(12, 720, 1440), meta=np.ndarray>
    RECO_std   (time, latitude, longitude) float64 dask.array<chunksize=(12, 720, 1440), meta=np.ndarray>
Attributes:
    description:  Upscaled global product of GPP and Reco (a combination of a...